In [24]:
import pandas as pd
from datasets import Dataset
import torch

In [25]:
# Load data
csv_path = 'data/book_genre_prediction.csv'
df = pd.read_csv(csv_path)
df

,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...
...,...,...,...,...
4652,4652,Hounded,fantasy,"Atticus O’Sullivan, last of the Druids, lives ..."
4653,4653,Charlie and the Chocolate Factory,fantasy,Charlie Bucket's wonderful adventure begins wh...
4654,4654,Red Rising,fantasy,"""I live for the dream that my children will be..."
4655,4655,Frostbite,fantasy,"Rose loves Dimitri, Dimitri might love Tasha, ..."


In [ ]:
PROMPT_TEMPLATE = """
### Instruction:
Summarize the following book.

### Input:
# Title: {title}

### Response:
"""

In [34]:
def make_prompt(row):
    return PROMPT_TEMPLATE.format(title=row['title'])

def make_target(row):
    return row['summary'] if isinstance(row['summary'], str) else ''

In [28]:
df = df.dropna(subset=['title', 'genre', 'summary'])
df['text'] = df.apply(lambda row: make_prompt(row) + ' ' + make_target(row), axis=1)
hf_dataset = Dataset.from_pandas(df[['text']])

In [32]:
df.iloc[0,4]

'\nSummarize the following book.\nTitle: Drowned Wednesday\n  Drowned Wednesday is the first Trustee among the Morrow Days who is on Arthur\'s side and wishes the Will to be fulfilled. She appears as a leviathan/whale and suffers from Gluttony. The book begins when Leaf is visiting Arthur and they are discussing the invitation that Drowned Wednesday sent him. Arthur had been admitted to hospital because of the damage done to his leg when he attempted to enter Tuesday\'s Treasure Tower. Suddenly, the hospital room becomes flooded with water as the two are transported to the Border Sea of the House. Leaf is snatched away by a large ship with green sails, known as the Flying Mantis, while Arthur remains in his bed. When the Medallion given him by the immortal called the Mariner apparently fails to summon help, Arthur is without hope. Eventually, a buoy marking the pirate Elishar Feverfew\'s treasure floats toward him. As soon as Arthur opens it, his hand is marked with a bloody red colour

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Hugging Face login and Mistral's policy acceptance is required to get access to Mistral
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model.to(device)

In [10]:
torch.cuda.memory_allocated()

0

In [52]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {"role": "user", "content": "Summarize the book twilight in a dramatic tone"},
]

prompt = """
### Background:
Your are a professional summarization-writing assistant. You got a job to write a summary. Your summary should start and finish in a way how summaries look like. Do not use bullet nor number lists. Do not start nor finish with greeting: just write a summary so it can be copy-pasted.

### Instruction
Summarize the following book using professional and funny tone.

### Input:
# Title: Harry Potter

### Response:
"""

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|system|>
You are a friendly chatbot.</s> 
<|user|>
Summarize the book twilight in a dramatic tone</s> 
<|assistant|>
This novel is undoubtedly a masterpiece, but let us take a dramatic turn on this unforgettable journey with the following synopsis:

With breathtaking plot twists and a mind-boggling narrative structure, author Stephenie Meyer's "Twilight" will leave you spellbound with its unique and captivating character development. This unforgettable novel chronicles the life of Bella Swan, a shy 17-year-old girl who lives in the idyllic town of Forks, Washington. Despite being raised in a small town, Bella falls in love with Edward Cullen, a mysterious and passionate vampire.

However, the story's heartbreak takes a turn as Bella faces the challenges of starting to live with the supernatural being, making her question her love for Edward and putting the entire Cullen family on edge. Along with these plot twists, the book features captivating and enchanting descriptions that captur

In [38]:
prompt = """### Instruction:
Summarize the following book in a gloomy, adult tone.

### Input:
Title: The Little Prince

### Response:
"""
ids = tokenizer(prompt, return_tensors="pt").to(model.device)
out = model.generate(**ids, max_new_tokens=120, temperature=0.7)
print(tokenizer.decode(out[0], skip_special_tokens=True))


/home/aniqu/anaconda3/envs/nlp-project/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


### Instruction:
Summarize the following book in a gloomy, adult tone.

### Input:
Title: The Little Prince

### Response:
The book is a gloomy and adult-themed tale about a young prince who embarks on a journey to find his lost love. The prince encounters various obstacles and encounters a variety of characters, including a wise old bird, a talking flower, and a group of animals. The story is told in a simple, yet poetic style, with vivid descriptions of the characters and their surroundings. The tone is somber and reflective, with a sense of melancholy and longing.


In [23]:
torch.cuda.memory_allocated()

4408715776